### Find the minimum eigenvalue of hamiltonian. ( hermitian matrix )

Example $\begin{bmatrix}1&2 \\ 3&4 \end{bmatrix}$

The matrix above is not hermitian, hence it needs to be converted to as below.

 $\begin{bmatrix}0&0&1&2 \\ 0&0&3&4 \\ 1&3&0&0 \\ 2&4&0&0 \end{bmatrix}$

In [90]:
from qiskit import *
from qiskit.algorithms.minimum_eigen_solvers import VQE
from qiskit.quantum_info import SparsePauliOp
import numpy as np
import scipy as sp
import itertools
import functools as ft
%matplotlib notebook
import matplotlib.pyplot as plt

import itertools

In [91]:
H = np.array([[0,0,1,2],
              [0,0,3,4],
              [1,3,0,0],
              [2,4,0,0]])

In [92]:
## ref : https://obliviateandsurrender.github.io/blogs/vqe.html
def decompose_ham_to_pauli(H):
    n = int(np.log2(len(H)))
    N = 2 ** n
    
    sI = np.eye(2, 2, dtype=complex)
    sX = np.array([[0, 1], [1, 0]], dtype=complex)
    sZ = np.array([[1, 0], [0,-1]], dtype=complex)
    sY = complex(0,-1)*np.matmul(sZ,sX)
    paulis = [sI, sX, sY, sZ]
    paulis_label = ['I', 'X', 'Y', 'Z']
    obs = []
    coeffs = []
    matrix = []
    for term in itertools.product(paulis, repeat=n):
        matrices = [pauli for pauli in term]
        coeff = np.trace(ft.reduce(np.kron, matrices) @ H) / N 
        coeff = np.real_if_close(coeff).item()
        # Hilbert-Schmidt-Product
        if not np.allclose(coeff, 0): 
            coeffs.append(coeff)
            obs.append(''.join([paulis_label[[i for i, x in enumerate(paulis)
            if np.all(x == t)][0]] for idx, t in enumerate(reversed(term))]))
            matrix.append(ft.reduce(np.kron, matrices))

    return obs, coeffs , matrix

In [93]:
o, amplitudes, _ = decompose_ham_to_pauli(H)

In [94]:
randHamiltonian = SparsePauliOp.from_list(list(zip(o, amplitudes)))
print(f"Number of qubits: {randHamiltonian.num_qubits}")

Number of qubits: 2


In [95]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.opflow import PauliSumOp

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=randHamiltonian)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

Reference value: -5.46499


In [98]:
# define ansatz and optimizer
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import SPSA

iterations = 125
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
spsa = SPSA(maxiter=iterations)

# define callback
# note: Re-run this cell to restart lists before training
counts = []
values = []

def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

# define Aer Estimator for noiseless statevector simulation
from qiskit.utils import algorithm_globals
from qiskit_aer.primitives import Estimator as AerEstimator

seed = 170
algorithm_globals.random_seed = seed

noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 3000},
    transpile_options={"seed_transpiler": seed},
)

In [99]:
# instantiate and run VQE
from qiskit.algorithms.minimum_eigensolvers import VQE

vqe = VQE(noiseless_estimator, ansatz, optimizer=spsa, callback=store_intermediate_result)
result = vqe.compute_minimum_eigenvalue(operator=randHamiltonian)

print(f"VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}")
print(
    f"Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}"
)

VQE on Aer qasm simulator (no noise): -5.44800
Delta from reference energy value is 0.01699
